In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
sns.set_palette('Dark2')


In [2]:
sns.set_style({'axes.axisbelow': True, 'axes.edgecolor': '.15', 'axes.facecolor': 'white',
               'axes.grid': True, 'axes.labelcolor': '.15', 'axes.linewidth': 0.5, 
               'figure.facecolor': 'white',  'grid.color': '.15',
               'grid.linestyle': '-', 'grid.alpha': .1, 'image.cmap': 'Greys', 
               'legend.frameon': False, 'legend.numpoints': 1, 'legend.scatterpoints': 1,
               'lines.solid_capstyle': 'round', 'axes.spines.right': False, 'axes.spines.top': False,  
               'text.color': '.15',  'xtick.top': False, 'ytick.right': False, 'xtick.color': '.15',
               'xtick.direction': 'out', 'xtick.major.size': 3, 'xtick.minor.size': 1.5,
               'ytick.color': '.15', 'ytick.direction': 'out', 'ytick.major.size': 3,'ytick.minor.size': 1.5})
sns.set_context('paper')

#http://phyletica.org/matplotlib-fonts/
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [3]:
from snapanalysis.config import OUTPUT_DIRECTORY as OUTPUT_DIRECTORY_MAIN
OUTPUT_DIRECTORY = os.path.join(OUTPUT_DIRECTORY_MAIN, 'scatterplots', 'annotated')
if not os.path.isdir(OUTPUT_DIRECTORY):
    os.makedirs(OUTPUT_DIRECTORY)

In [4]:
from contextlib import contextmanager
from rpy2.robjects.lib import grdevices
from IPython.display import Image, display

from rpy2.robjects import pandas2ri, numpy2ri
from rpy2 import robjects
from rpy2.robjects.packages import importr
import rpy2.robjects.lib.ggplot2 as r_ggplot2
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage

r_ggrepel = importr('ggrepel')
r_print = robjects.r['print']
r_grid = importr('grid')
r_base = importr('base')

/usr/local/lib/python3.6/site-packages/rpy2/robjects/lib/ggplot2.py:69: UserWarning: This was designed againt ggplot2 version 3.1.0 but you have 3.1.1
  'have %s' % (TARGET_VERSION, ggplot2.__version__))


In [5]:
r_tick_formatter_code = """
formatterFunOneDigit <- function(x) sprintf("%.1f", x)
formatterFunTwoDigits <- function(x) sprintf("%.2f", x)
"""

r_custom = SignatureTranslatedAnonymousPackage(r_tick_formatter_code, "custom")

@contextmanager
def r_inline_plot(width=4, height=4, dpi=100):

    with grdevices.render_to_bytesio(grdevices.png, 
                                     width=width,
                                     height=height, 
                                     units='in',
                                     res=dpi) as b:

        yield

    data = b.getvalue()
    display(Image(data=data, format='png', embed=True))
    

@contextmanager
def r_plot_to_pdf(filename, width=4, height=4):
    grdevices.pdf(filename, width=width, height=height)
    yield
    grdevices.dev_off()


In [6]:
from snapanalysis.models.enrichment.generate import OUTPUT_FILE as ENRICHMENT_FILE
from snapanalysis.models.enrichment.generate import MATRIX_COLUMN_FORWARD, MATRIX_COLUMN_REVERSE
MATRIX_COLUMN_FORWARD

'Ratio H/L normalized (log2) (adjusted, imputed, forward)'

In [7]:
with pd.HDFStore(ENRICHMENT_FILE, 'r') as store:
    enrichment_data = store['enrichment_data']

In [8]:
from snapanalysis.preprocessing.pulldown_metadata import OUTPUT_FILE as META_FILE
with pd.HDFStore(META_FILE, 'r') as store:
    PULL_DOWN_NAMES = store['/meta/names_and_types']['Pull-Down name']

In [9]:
WIDTH = HEIGHT =  2.6 # in

In [10]:
import parameters

import importlib
importlib.reload(parameters)

<module 'parameters' from '/notebooks/scatterplots/parameters.py'>

In [11]:
COLOUR_BACKGROUND = parameters.COLOUR_BACKGROUND

In [12]:
import matplotlib.ticker as ticker

In [13]:
def plottable_data_for_pulldown(pulldown, 
                                ratio_mode='processed',
                                hide_imputed=False,
                                fdr_threshold_for_label=1.0, 
                                std_threshold_for_label=1.0,
                                max_labels=10):
    data = enrichment_data.swaplevel()
    
    if ratio_mode == 'processed':
        col_x, col_y = MATRIX_COLUMN_FORWARD, MATRIX_COLUMN_REVERSE
    elif ratio_mode == 'raw':
        col_x, col_y = 'Ratio H/L (log2) (forward)', 'Ratio H/L (log2) (reverse)'
    else:
        raise ValueError(f'Unknown ratio mode {ratio_mode!r}')
        
    
    data = data.loc[pulldown].dropna(subset=[col_x, col_y]).copy()
    data['imputed'] = ~data['Imputation type'].isnull()
    
    if hide_imputed:
        data = data[~data['imputed']]
        
    
    data = data[[col_x, col_y, 'imputed']]
    data = data.rename(columns={col_x: 'x', col_y: 'y', 'is_imputed': 'imputed'})
    
    data['neg_y'] = data['y'] * -1
    
    data['mean_xy'] = data[['x', 'neg_y']].mean(axis=1)
    data['mean_xy_std'] = data[['x', 'neg_y']].std(axis=1)
    
    data['abs_mean_xy'] = data['mean_xy'].abs()
    
    labellable_data = data[(data['mean_xy'].abs() >= fdr_threshold_for_label) & (data['mean_xy_std'] <= std_threshold_for_label)]
    
    order_asc = labellable_data[labellable_data['mean_xy'] < 0].sort_values(by='mean_xy', ascending=True).index
    order_desc = labellable_data[labellable_data['mean_xy'] > 0].sort_values(by='mean_xy', ascending=False).index
    
    n_labelled = 0
    index_asc = 0
    index_desc = 0
    
    data['label'] = ''

    turn = 'asc'
    
    while (n_labelled < max_labels) & ((index_asc < len(order_asc)) | (index_desc < len(order_desc))):
        
        if turn == 'asc':
            if index_asc >= len(order_asc):
                ix = None
            else:
                ix = order_asc[index_asc]
            index_asc += 1
            turn = 'desc'
            
        elif turn == 'desc':
            if index_desc >= len(order_desc):
                ix = None
            else:
                ix = order_desc[index_desc]
            
            index_desc += 1
            turn = 'asc'
            
        if ix is None:
            continue
        
        data.loc[ix, 'label'] = ix
        n_labelled += 1
    
    data = data.sort_values(by='abs_mean_xy')
    data = data.reset_index()
    
    with robjects.conversion.localconverter(robjects.default_converter + pandas2ri.converter):
        r_data = pandas2ri.py2rpy(data)
    return data, r_data

In [14]:
def plot_scatter(pull_down,
                 xlim=None,
                 ylim=None,
                 ratio_mode='processed',
                 **kwargs):
        
    base_size = 10
    label_size = 3
    
    data, r_data = plottable_data_for_pulldown(pull_down, ratio_mode=ratio_mode, **kwargs)
    
    plot = r_ggplot2.ggplot(r_data)
    plot += r_ggplot2.theme_minimal(base_size = base_size)
    
    plot += r_ggplot2.theme(**{'panel.grid.major': r_ggplot2.element_blank(), 
                               'panel.grid.minor': r_ggplot2.element_blank(),
                               'panel.border':  r_ggplot2.element_rect(fill=robjects.rinterface.NA,
                                                                       color = "black")})
    plot += r_ggplot2.theme(text=r_ggplot2.element_text(family='Helvetica', face='plain'))
    plot += r_ggplot2.theme(**{'plot.title': r_ggplot2.element_text(hjust=0.5),
#                               'axis.title.y': r_ggplot2.element_text((t = 0, r = 20, b = 0, l = 0)),
                              })

    aes_points = r_ggplot2.aes_string(x='x', y='y', shape='imputed')
    plot += aes_points
    
    if xlim is not None:
        plot += r_ggplot2.scale_x_continuous(labels=r_custom.formatterFunTwoDigits, limits=robjects.r.c(*xlim))
    else:
        plot += r_ggplot2.scale_x_continuous(labels=r_custom.formatterFunTwoDigits)
        
    if ylim is not None:
        plot += r_ggplot2.scale_y_continuous(labels=r_custom.formatterFunTwoDigits, limits=robjects.r.c(*ylim))
    else:
        plot += r_ggplot2.scale_y_continuous(labels=r_custom.formatterFunTwoDigits)
         
    plot += r_ggplot2.geom_point(alpha=0.5)

    aes_text = r_ggplot2.aes_string(label='label')
    plot += aes_text
    plot += r_ggrepel.geom_text_repel(aes_text, 
                                       size=label_size,
                                       family='Helvetica',
                                       **{'show.legend': False, 
                                          'point.padding': 0.25, 
                                          'min.segment.length': 0,
                                          'segment.color': '#BDBDBD'},
                                     )
    
    
    plot += r_ggplot2.geom_hline(yintercept=0, 
                                 color='black', alpha=1)
    
    plot += r_ggplot2.geom_vline(xintercept=0, 
                                 color='black', alpha=1)
   
    other_text = '' if ratio_mode == 'processed' else ' (raw data)'
    plot += r_ggplot2.labs(x='Ratio H/L (log2, forward)', 
                           y='Ratio H/L (log2, reverse)', 
                           title='{}: {}{}'.format(pull_down, PULL_DOWN_NAMES.loc[pull_down], other_text)
                          )
    
        
    plot.plot()
    

In [15]:
pulldowns = enrichment_data.reset_index()['Pull-Down ID'].unique()

In [16]:
pulldowns

array(['H01', 'H01M', 'H02', 'H03', 'H03M', 'H04', 'H04M', 'H05', 'H06',
       'H07', 'H07M', 'H08', 'H08M', 'H09', 'H10', 'H11', 'H12', 'H13',
       'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 'H21', 'H22',
       'H23', 'H24', 'H25', 'H26', 'H27M', 'H28', 'H29', 'H30', 'H31',
       'H32', 'H33', 'H34', 'H35', 'H36', 'H37', 'H38', 'H39', 'H39M',
       'H40', 'H41', 'H42', 'H43', 'H44', 'H45', 'H46', 'H46M', 'H47',
       'H47M'], dtype=object)

In [26]:
width = 5
height = 4

for pulldown in tqdm(pulldowns):
    with r_plot_to_pdf(os.path.join(OUTPUT_DIRECTORY, f'{pulldown}-processed.pdf'),
                       width=width, height=height):
            plot_scatter(pulldown)
            
    with r_plot_to_pdf(os.path.join(OUTPUT_DIRECTORY, f'{pulldown}-processed-not-imputed.pdf'),
                       width=width, height=height):
            plot_scatter(pulldown, hide_imputed=True)
            
    
    with r_plot_to_pdf(os.path.join(OUTPUT_DIRECTORY, f'{pulldown}-raw.pdf'),
                       width=width, height=height):
            plot_scatter(pulldown, ratio_mode='raw')
            
#     break

100%|██████████| 55/55 [01:36<00:00,  1.77s/it]


In [27]:
pulldown = 'H02'

with r_plot_to_pdf(os.path.join(OUTPUT_DIRECTORY, f'{pulldown}-processed-not-imputed-fixed-xlim.pdf'), 
                   width=width, height=height):
    plot_scatter(pulldown, hide_imputed=True, xlim=(-2, 4), ylim=(-4, 2))



In [28]:
pulldown = 'H05'

with r_plot_to_pdf(os.path.join(OUTPUT_DIRECTORY, f'{pulldown}-processed-not-imputed-fixed-xlim.pdf'), 
                   width=width, height=height):
    plot_scatter(pulldown, hide_imputed=True, xlim=(-2, 4), ylim=(-4, 2))